In [1]:
from tqdm.auto import tqdm, trange
import time
import pickle
import os
from seleniumbase import Driver as SeleniumBaseDriver
from wifi import Wifi

def get_seleniumbase_driver(eager=True, **kwargs):
    # proxy = "SERVER:PORT" or "USER:PASS@SERVER:PORT"
    driver = SeleniumBaseDriver(
        browser="chrome",
        uc=True,
        locale_code="en",
        page_load_strategy="eager" if eager else None,
        **kwargs,
    )
    # driver.maximize_window()

    return driver

In [2]:
wifi = Wifi()
ans = wifi.restart()

In [3]:
driver: SeleniumBaseDriver = get_seleniumbase_driver()

In [4]:
if os.path.exists("docs.pkl"):
    with open("docs.pkl", "rb") as f:
        docs = pickle.load(f)
        all_hrefs = set([doc["href"] for doc in docs])
else:
    docs = []
    all_hrefs = set()

for page_num in trange(1, 12):
    driver.get(f"https://company.rzd.ru/ru/9353/page/105103?f3174_pagesize=60&&rubrics=&doc_num=&date_end=&doc_type=&source=&f_sortcol=&f_sortdir=&date_begin=&doc_name=&text_search_type=0&text_search=&f3174_pagenumber={page_num}")

    table = driver.find_element(by="css selector", value="table.table")

    links = table.find_elements(by="css selector", value="tr > td > a")
    
    hrefs = [link.get_attribute("href") for link in links]

    for href in hrefs:
        if href not in all_hrefs:
            all_hrefs.add(href)
            docs.append({
                "html": None,
                "href": href
            })

    if page_num % 5 == 0:
        wifi = Wifi()
        wifi.restart()

len(docs)

  0%|          | 0/11 [00:00<?, ?it/s]

640

In [6]:
driver.close()

In [7]:
with open("docs.pkl", "wb") as f:
    pickle.dump(docs, f)

# Hrefs

In [28]:
with open("docs.pkl", "rb") as f:
    docs = pickle.load(f)

len([doc['html'] for doc in docs if doc['html'] is not None])

640

In [25]:
wifi = Wifi()
ans = wifi.restart()

In [26]:
driver: SeleniumBaseDriver = get_seleniumbase_driver()
driver.set_page_load_timeout(10)
driver.implicitly_wait(10)

In [27]:
i = 0

exception_counter = 0

with tqdm(total=len(docs), desc="Процесс выполнения") as pbar:
    while i < len(docs):
        doc = docs[i]
        href = docs[i]["href"]
        if doc["html"] is not None:
            pbar.update(1)
            i += 1
            continue

        try:
            # print(href)
            driver.get(href)
            main = driver.find_element(by="css selector", value="div.docs")
            html = main.get_attribute("outerHTML")

        except KeyboardInterrupt:
            break
        except Exception as e:
            exception_counter += 1
            print(f"Exception {exception_counter},", e)
            # time.sleep(5)
            wifi = Wifi()
            ans = wifi.restart()
            continue

        docs[i]["html"] = html

        with open("docs.pkl", "wb") as f:
            pickle.dump(docs, f)

        pbar.update(1)
        i += 1
        time.sleep(1)

        if i % 8 == 7:
            wifi = Wifi()
            wifi.restart()



Процесс выполнения:   0%|          | 0/640 [00:00<?, ?it/s]

Exception 1, Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.docs"}
  (Session info: chrome=129.0.6668.100); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF60A9EB675+29573]
	(No symbol) [0x00007FF60A9604A0]
	(No symbol) [0x00007FF60A81B6EA]
	(No symbol) [0x00007FF60A86F815]
	(No symbol) [0x00007FF60A86FA6C]
	(No symbol) [0x00007FF60A8BB917]
	(No symbol) [0x00007FF60A89733F]
	(No symbol) [0x00007FF60A8B86BC]
	(No symbol) [0x00007FF60A8970A3]
	(No symbol) [0x00007FF60A8612DF]
	(No symbol) [0x00007FF60A862441]
	GetHandleVerifier [0x00007FF60AD1C5BD+3375821]
	GetHandleVerifier [0x00007FF60AD679B7+3684039]
	GetHandleVerifier [0x00007FF60AD5CDDB+3640043]
	GetHandleVerifier [0x00007FF60AAAB7F6+816390]
	(No symbol) [0x00007FF60A96B7AF]
	(No symbol) [0x00007FF60A9675D4]
	(No symbol) [0x00007FF60A967770]
	(No symbo

In [ ]:
driver.close()